In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df_iris = pd.read_csv('iris.csv')
df_iris

,ID,sepallength,sepalwidth,petallength,petalwidth,class
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
#Exploração do dataframe
df_iris.shape

(150, 6)

Add descriptive features

In [3]:
# É necessário transformar as colunas numéricas em categórias (baixa, média, alta), o que é necessário para o algoritmo ID3, que trabalha apenas com atributos categóricos.

df_iris['sepal_length_bin'] = pd.cut(df_iris['sepallength'], bins=3, labels=["Low", "Medium", "High"])
df_iris['sepal_width_bin']  = pd.cut(df_iris['sepalwidth'],  bins=3, labels=["Low", "Medium", "High"])
df_iris['petal_length_bin'] = pd.cut(df_iris['petallength'], bins=3, labels=["Low", "Medium", "High"])
df_iris['petal_width_bin']  = pd.cut(df_iris['petalwidth'],  bins=3, labels=["Low", "Medium", "High"])


In [20]:
df_iris.head()

,ID,sepallength,sepalwidth,petallength,petalwidth,class,sepal_length_bin,sepal_width_bin,petal_length_bin,petal_width_bin
0,1,5.1,3.5,1.4,0.2,Iris-setosa,Low,Medium,Low,Low
1,2,4.9,3.0,1.4,0.2,Iris-setosa,Low,Medium,Low,Low
2,3,4.7,3.2,1.3,0.2,Iris-setosa,Low,Medium,Low,Low
3,4,4.6,3.1,1.5,0.2,Iris-setosa,Low,Medium,Low,Low
4,5,5.0,3.6,1.4,0.2,Iris-setosa,Low,Medium,Low,Low


<h1/>2 - Split the features and the target</h1>

In [4]:
#Separar as colunas com os valores discretos da coluna cujo valor se pretende prever
X_iris = df_iris.drop(columns=["ID","class", "sepallength", "sepalwidth", "petallength", "petalwidth"])
y_iris = df_iris['class']

In [24]:
X_iris.head()

,sepal_length_bin,sepal_width_bin,petal_length_bin,petal_width_bin
0,Low,Medium,Low,Low
1,Low,Medium,Low,Low
2,Low,Medium,Low,Low
3,Low,Medium,Low,Low
4,Low,Medium,Low,Low


In [25]:
y_iris.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: class, dtype: object

In [5]:
# Definir uma seed para garantir resultados reproduzíveis
seed = 42

# Baralhar as linhas do DataFrame
df_shuffled = df_iris.sample(frac=1, random_state=seed).reset_index(drop=True)

# Definir a proporção de teste (20%) e calcular o índice de corte
test_frac = 0.2
cut = int(len(df_shuffled) * (1 - test_frac))

# Separar os dados em treino (80%) e teste (20%)
df_iris_train = df_shuffled.iloc[:cut]
df_iris_test  = df_shuffled.iloc[cut:]

feature_cols = ['sepal_length_bin',
                'sepal_width_bin',
                'petal_length_bin',
                'petal_width_bin']

# Separar o dataframe em treino e teste
X_iris_train = df_iris_train[feature_cols]
y_iris_train = df_iris_train['class']

X_iris_test  = df_iris_test[feature_cols]
y_iris_test  = df_iris_test['class']



In [6]:
def calculate_entropy(labels: pd.Series) -> float:
    # Calcula a entropia de uma coluna
    counts = labels.value_counts()
    probs = counts / len(labels)

    entropy = -sum(x * math.log2(x) for x in probs)

    return entropy


def information_gain(data: pd.DataFrame, feature: str, target_column: str) -> float:
    # Calcula o ganho de informação ao dividir os dados por um determinado atributo

    total_entropy = calculate_entropy(data[target_column])  # Entropia total antes da divisão

    values = data[feature].unique()  # Todos os valores distintos da feature
    weighted_entropy = 0

    for value in values:
        subset = data[data[feature] == value]
        weight = len(subset) / len(data)
        subset_entropy = calculate_entropy(subset[target_column])
        weighted_entropy += weight * subset_entropy

    gain = total_entropy - weighted_entropy  # Ganho de informação
    return gain


def id3(data: pd.DataFrame, target_column: str, features: list) -> dict:

    # Caso base: todos os rótulos são iguais → retorna a classe
    if len(data[target_column].unique()) == 1:
        return data[target_column].iloc[0]

    # Caso base: não há mais atributos → retorna a classe mais comum
    if len(features) == 0:
        return data[target_column].mode()[0]

    # Escolher o melhor atributo com maior ganho de informação
    gains = {feature: information_gain(data, feature, target_column) for feature in features}
    best_feature = max(gains, key=gains.get)

    tree = {best_feature: {}}  # Inicializa o nó da árvore

    # Para cada valor do melhor atributo, constrói a subárvore recursivamente
    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]

        if subset.empty:
            # Se o subconjunto estiver vazio, retorna a classe mais comum
            tree[best_feature][value] = data[target_column].mode()[0]
        else:
            # Chamada recursiva com o subconjunto e os restantes atributos
            new_features = [f for f in features if f != best_feature]
            subtree = id3(subset, target_column, new_features)
            tree[best_feature][value] = subtree

    return tree

tree = id3(df_iris, 'class', feature_cols)


In [8]:
tree

{'petal_width_bin': {'Low': 'Iris-setosa',
  'Medium': {'petal_length_bin': {'Medium': {'sepal_length_bin': {'High': 'Iris-versicolor',
      'Medium': 'Iris-versicolor',
      'Low': {'sepal_width_bin': {'Low': 'Iris-versicolor',
        'Medium': 'Iris-versicolor'}}}},
    'High': {'sepal_length_bin': {'Medium': {'sepal_width_bin': {'Medium': 'Iris-versicolor',
        'Low': 'Iris-virginica'}},
      'High': 'Iris-virginica'}}}},
  'High': {'petal_length_bin': {'Medium': {'sepal_width_bin': {'Medium': {'sepal_length_bin': {'Medium': 'Iris-virginica'}},
      'Low': 'Iris-virginica'}},
    'High': 'Iris-virginica'}}}}

Test

In [9]:
def classify(example, tree):
    if not isinstance(tree, dict):
        return tree  # If it's a label, return it

    # Otherwise, get the feature at this node
    feature = next(iter(tree))
    feature_value = example[feature]

    # Move to the subtree for this value
    subtree = tree[feature].get(feature_value)

    if subtree is None:
        return None  # Value not seen during training

    return classify(example, subtree)


In [10]:
df_iris_test.loc[:, 'prediction'] = X_iris_test.apply(lambda row: classify(row, tree), axis=1)
accuracy = (df_iris_test['prediction'] == y_iris_test).mean()
print(f"Test accuracy: {accuracy:.2%}")

Test accuracy: 96.67%


/tmp/ipykernel_33560/617801768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iris_test.loc[:, 'prediction'] = X_iris_test.apply(lambda row: classify(row, tree), axis=1)


In [12]:
df_c4 = pd.read_csv('DataFrame_connectfour.csv')
df_c4.head()

,Posição 1,Posição 2,Posição 3,Posição 4,Posição 5,Posição 6,Posição 7,Posição 8,Posição 9,Posição 10,...,Posição 35,Posição 36,Posição 37,Posição 38,Posição 39,Posição 40,Posição 41,Posição 42,Melhor jogada,Probabilidade de vitória (%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,19.286967
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,3,29.893868
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,2,16.447545
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,X,X,NaN,NaN,NaN,1,57.776920
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,O,X,X,NaN,NaN,NaN,2,27.622015


In [13]:
df_c4.fillna(".", inplace=True)
df_c4.head()

,Posição 1,Posição 2,Posição 3,Posição 4,Posição 5,Posição 6,Posição 7,Posição 8,Posição 9,Posição 10,...,Posição 35,Posição 36,Posição 37,Posição 38,Posição 39,Posição 40,Posição 41,Posição 42,Melhor jogada,Probabilidade de vitória (%)
0,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,3,19.286967
1,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,X,.,.,.,3,29.893868
2,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,X,.,.,.,2,16.447545
3,.,.,.,.,.,.,.,.,.,.,...,.,.,.,X,X,.,.,.,1,57.776920
4,.,.,.,.,.,.,.,.,.,.,...,.,.,O,X,X,.,.,.,2,27.622015


In [14]:
# Separar as colunas com cada posição do tabuleiro da coluna cujo valor se pretende prever
feature_cols = [col for col in df_c4.columns if col.startswith("Pos")]
target_col = "Melhor jogada"

seed = 42
df_shuffled = df_c4.sample(frac=1, random_state=seed).reset_index(drop=True)

# Separar o dataframe em treino e teste
test_frac = 0.2
cut = int(len(df_shuffled) * (1 - test_frac))
df_train = df_shuffled.iloc[:cut]
df_test  = df_shuffled.iloc[cut:]

# Treinar a árvore
tree = id3(df_train, target_col, feature_cols)


# Aplicar o algoritmo ao dataset de teste
df_test = df_test.copy()
df_test['prediction'] = df_test[feature_cols].apply(lambda row: classify(row, tree), axis=1)


accuracy = (df_test['prediction'] == df_test[target_col]).mean()
print(f"Test accuracy: {accuracy:.2%}")


Test accuracy: 84.40%
